<a href="https://colab.research.google.com/github/dpatinoecheverri/ENV717_S23/blob/main/Assignments/A09/ENV717_A9_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 9
##ENV717- Spring 2023
---

Instructor: [Dalia Patino-Echeverri](mailto:dalia.patino@duke.edu)

Author(s): [Dalia Patino-Echeverri](mailto:dalia.patino@duke.edu), [Mauricio Hernandez](mailto:mmh54@duke.edu), [Abbhijith Hari](mailto:ah580@duke.edu)


[Github repository](https://github.com/dpatinoecheverri/ENV717_S23)

#2. (15%) 

Use excel-solver to find a solution to the LP formulated in 1, and report the following:

a. Pi = net power injection at each bus (negative means there is a net power withdrawal), i=1,2,3. <br>
b. Gg: power generation from each generator g, g=1,2,3,4. <br>
c. fij: power flow on the transmission line connecting node i to node j, in the direction from i to j. (negative means the power flows in the opposite direction).<br>
d. LMPi: Locational Marginal Price at bus i. (\$/MWh)<br>
e. Merchandising surplus in \$.<br>
f. Interpretation of the allowable increase and decrease for the LMPs of each bus


![three bus system](https://drive.google.com/uc?export=view&id=1-00sJ7wEGDg_CoDKqwTxwF91meBIysZY)

In [92]:
import shutil
import sys
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

!pip install pyomo
#4. Install the solver
!apt-get install -y -qq glpk-utils

from pyomo.environ import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')
#save path
#access path where notebook is stored
path = "/content/gdrive/MyDrive/ENV717-S2023/A09/"
#Change the working directory to the folder
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [94]:
NumNodes = 3
NumLines = 3
NumGens = 4

CostMWH = [12, 15, 10, 8]
MaxGenCapac = [150, 200, 150, 400]
#Gens = ['GA', 'GB', 'GC', 'GD']
G=np.array([g for g in range(0,NumGens)]) 
N=np.array([n for n in range(0,NumNodes)])
L=np.array([l for l in range(0,NumLines)])
Demand = [400, 80, 40]

LineFromBus = np.array([[1,0,0],[0,1,0], [0,0,1]])
LineToBus = np.array([[0,1,0],[0,0,1], [1,0,0]])
GeneratorInBus = np.array([[0,1,0], [1,0,0], [0,0,1], [0,0,1]])
LineReactance = np.array([0.2, 0.3, 0.3])
LineCapacity = np.array([250, 250, 250])

### Defining the Economic Dispatch Model

In [95]:
#optimization model definition
def ed_model():
  lp=ConcreteModel()
  #Declaring Parameters
  lp.Cost = Set(initialize = CostMWH)
  lp.Gen = Set(initialize = G)
  lp.L=Set(initialize=L)
  lp.N=Set(initialize=N)
  lp.dual = Suffix(direction=Suffix.IMPORT) #Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect

  #Declaring Decision Variables
  lp.p = Var(lp.Gen, domain= PositiveReals)
  lp.flow=Var(lp.L, bounds = (-max(LineCapacity), max(LineCapacity))) #This is power flow on a line
  #Constraints
  lp.MaxGenConst = Constraint(lp.Gen, rule=lambda lp, g:  lp.p[g] <= MaxGenCapac[g])
  #Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
  lp.KVLAroundLoopConstraint=Constraint(rule=lambda lp: sum(LineReactance[l]*lp.flow[l] for l in L)==0) 
  lp.NodePowerBalanceConstraint=Constraint(lp.N, rule=lambda lp, n: sum(LineToBus[l,n]*lp.flow[l] for l in L) - 
                                           sum(LineFromBus[l,n]*lp.flow[l] for l in L) + 
                                           sum(GeneratorInBus[g,n]*lp.p[g] for g in G) == Demand[n])
  lp.MaxFlow=Constraint(lp.L, rule=lambda lp, l:  lp.flow[l] <= LineCapacity[l])
  lp.MaxCounterFlow=Constraint(lp.L, rule=lambda lp, l:  lp.flow[l] >= -LineCapacity[l])
  #Objective Function
  lp.system_cost=Objective(expr=sum(lp.p[g]*CostMWH[g] for g in lp.Gen), sense=minimize)

  return lp

### Solving the optimization problem

In [96]:
m = ed_model()

SolverFactory('glpk', executable='/usr/bin/glpsol').solve(m).write() # Instead of this line, write "opt.solve(m)" if model's details are not important

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4560.0
  Upper bound: 4560.0
  Number of objectives: 1
  Number of constraints: 15
  Number of variables: 8
  Number of nonzeros: 24
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01616358757019043
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [97]:
print('SOLUTION')
print('System cost = ${:}'.format(m.system_cost()))
print("--------------------------------------------")
print('Power generated by unit in MW')
for g in G:
  print("generator {0:2d} = {1:.0f} ".format(g+1, m.p[g]()))
print("--------------------------------------------")
print('Flow on transmission lines in MW')

for l in L:
  print("line {0:2d} = {1:.0f} ".format(l+1, m.flow[l]()))
print("--------------------------------------------")
print('LMPs in $/MWh')
for n in N:
  print("Node {0:2d} = {1:.2f}".format(n+1, m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION
System cost = $4560.0
--------------------------------------------
Power generated by unit in MW
generator  1 = 80 
generator  2 = 0 
generator  3 = 40 
generator  4 = 400 
--------------------------------------------
Flow on transmission lines in MW
line  1 = -150 
line  2 = -150 
line  3 = 250 
--------------------------------------------
LMPs in $/MWh
Node  1 = 13.33
Node  2 = 12.00
Node  3 = 10.00


In [99]:
%%shell
jupyter nbconvert --to html "/content/gdrive/MyDrive/ENV717-S2023/A09/ENV717_A9_Solution.ipynb"

[NbConvertApp] Converting notebook /content/gdrive/MyDrive/ENV717-S2023/A09/ENV717_A9_Solution.ipynb to html
[NbConvertApp] Writing 605957 bytes to /content/gdrive/MyDrive/ENV717-S2023/A09/ENV717_A9_Solution.html
